# Effective label choosing strategy based on inter-class similarities

Generation of the Least Similar and the Most similar classes to the ground truth labels. Such labels can be used as target labels in targeted and also in gradient-based non-targeted attacks (when minimization is used to compute the image perturbations). 

Class Similarity Matrix based on example model's weights.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.applications.imagenet_utils import decode_predictions
from nltk.corpus import wordnet as wn
import nltk
from tensorflow.keras.applications.imagenet_utils import decode_predictions
from keras.utils import to_categorical
from sklearn.metrics.pairwise import cosine_similarity

# example model from Keras Applications (other models are compatible):
from tensorflow.keras.applications import MobileNetV2

Below, we create the Class Similarity Matrix based on example model's weights for ImageNet classes.

In [2]:
# reading a model:
model = MobileNetV2(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

class_i = np.moveaxis(model.layers[-1].get_weights()[0], 0, -1)
class_j = np.moveaxis(model.layers[-1].get_weights()[0], 0, -1)
cs_tab = cosine_similarity(class_i, class_j)

# cs_tab - Class Similarity Matrix generated based on model's weights

Below, we define lists with MS and LS labels for ImageNet. They can be used to find target classes for targeted and non-targeted attacks. Label at index k in each (MS and LS) table corresponds to the most and the least similar class to class k (according to ImageNet labeling - with increasing WordNet 3.0 ids).

In [3]:
MS_labels = np.argsort(-cs_tab)[:, 1]
LS_labels = np.argsort(-cs_tab)[:, -1]

Reading the MS and LS target labels based on Sorted Class Similarity Matrix:

In [4]:
example_label = 3
example_labels_MS_class = MS_labels[example_label]
example_labels_LS_class = LS_labels[example_label]

Ground truth label:

In [5]:
decode_predictions(to_categorical(np.expand_dims(example_label, axis=-1), num_classes=1000), top=1)

[[('n01491361', 'tiger_shark', 1.0)]]

The Most Similar target label (MS label):

In [6]:
decode_predictions(to_categorical(np.expand_dims(example_labels_MS_class, axis=-1), num_classes=1000), top=1)

[[('n01484850', 'great_white_shark', 1.0)]]

The Least Similar target label (LS label):

In [7]:
decode_predictions(to_categorical(np.expand_dims(example_labels_LS_class, axis=-1), num_classes=1000), top=1)

[[('n01667778', 'terrapin', 1.0)]]

Generating target labels for ground truth labels in a non-batch form (1D numpy array):

In [8]:
ground_truth_labels = np.array([1,3,5])

In [9]:
MS_labels[ground_truth_labels]

array([392,   2,   6], dtype=int64)

Generating target labels for ground truth labels in a batch form:

In [10]:
ground_truth_labels = np.expand_dims(np.array([1,3,5]), axis=1)

In [11]:
MS_labels[ground_truth_labels]

array([[392],
       [  2],
       [  6]], dtype=int64)